In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# loading libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid',{"grid.color": ".6", "grid.linestyle": ":"})
%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 7)

In [ ]:
from google.cloud import bigquery

# creating client object (because it holds project and a connection to the bigquery services)

In [ ]:
client=bigquery.Client()

# creating a chicago_crime dataset object

In [ ]:
chicago_crime_ref=client.dataset("chicago_crime", project="bigquery-public-data")

# API request to fetch the chicago_crime dataset

In [ ]:
chicago_crime=client.get_dataset(chicago_crime_ref)

# creaing a crime table object

In [ ]:
crime_ref=chicago_crime_ref.table('crime')

# API request to fetch crime table

In [ ]:
crime=client.get_table(crime_ref)

# checking schema

In [ ]:
crime.schema

# loading data in to dataframe with limited numbers

In [ ]:
first10_rows=client.list_rows(crime,max_results=10).to_dataframe()
first10_rows

# solving two queries one by one

# **What categories of crime exhibited the greatest year-over-year increase between 2015 and 2016?


In [ ]:
query="""
    SELECT primary_type, year
    FROM `bigquery-public-data.chicago_crime.crime`
    WHERE year in (2015,2016)
    """
dry_run_config=bigquery.QueryJobConfig(dry_run=True)

dry_run_query_job=client.query(query,job_config=dry_run_config)

print(f"the query will be proccessed in {dry_run_query_job.total_bytes_processed} bytes")

In [ ]:
one_gb=1000 *1000 * 1000

safe_config=bigquery.QueryJobConfig(maximum_bytes_billed=one_gb)

safe_query_job=client.query(query,job_config=safe_config)

crime_cat=safe_query_job.to_dataframe()


In [ ]:
crime_cat_groupby=crime_cat.groupby('year')['primary_type'].value_counts().sort_values(ascending=False)

In [ ]:
crime_cat_groupby_temp=crime_cat_groupby[:50]

In [ ]:
crime_cat_groupby_temp=crime_cat_groupby_temp.reset_index(level=[0,1],name=('year','primary_type'))

In [ ]:
crime_cat_groupby_temp.columns=['year','primary_type','freuency']

# viewing the results in a dataframe with a small number of values

In [ ]:
crime_cat_groupby_temp[:10]

# visualizing the query output 

In [ ]:
sns.lineplot(data=crime_cat_groupby_temp,x='primary_type',y='freuency',hue='year')
plt.xticks(rotation=90)
plt.show()

# output

# By adding 4268 additional instances from 2015 to 2016, theft rose

# now second query

# **Which month generally has the greatest number of motor vehicle thefts?

In [ ]:
query="""
    SELECT EXTRACT(month FROM DATE) AS month,primary_type
    FROM `bigquery-public-data.chicago_crime.crime`
    WHERE primary_type='MOTOR VEHICLE THEFT'
    """
dry_run_config=bigquery.QueryJobConfig(dry_run=True)

dry_run_query_job=client.query(query,job_config=dry_run_config)

print(f"the query will be proccessed in {dry_run_query_job.total_bytes_processed} bytes")

In [ ]:
one_gb=1000 *1000 * 1000

safe_config=bigquery.QueryJobConfig(maximum_bytes_billed=one_gb)

safe_query_job=client.query(query,job_config=safe_config)

crime_cat=safe_query_job.to_dataframe()

In [ ]:
crime_cat_month=crime_cat.groupby('month')['primary_type'].value_counts().sort_values(ascending=False)

In [ ]:
crime_cat_month

In [ ]:
crime_cat_month=crime_cat_month.reset_index(level=[0,1],name=('month','primary_type'))

In [ ]:
crime_cat_month.columns=['month','primary_type','freuency']

# viewing the results in a dataframe with a small number of values second time

In [ ]:
crime_cat_month

# visualizing the query output second time

In [ ]:
ax=sns.pointplot(data=crime_cat_month,x='month',y='freuency',hue='month')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()

# output 

# Most motor vehicle thefts occur in the month of October

# Please vote for me and comment if you like my work so it allows me to improve on it.